In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Assuming you have your training and testing data in 'train_data' and 'test_data'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# Load data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [3]:
# Extracting Title from the Name column
train_data['Title'] = train_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
test_data['Title'] = test_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [4]:
# Handle missing values for numerical columns
numerical_cols = ['Age', 'Fare']
train_data[numerical_cols] = train_data[numerical_cols].fillna(train_data[numerical_cols].mean())
test_data[numerical_cols] = test_data[numerical_cols].fillna(test_data[numerical_cols].mean())

In [5]:
# Handle missing values for categorical columns
categorical_cols = ['Pclass', 'Sex', 'Embarked', 'Title']
train_data[categorical_cols] = train_data[categorical_cols].fillna(train_data[categorical_cols].mode().iloc[0])
test_data[categorical_cols] = test_data[categorical_cols].fillna(test_data[categorical_cols].mode().iloc[0])

In [6]:
# Combine family members
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1

In [7]:
# Encode categorical features
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    train_data[col] = label_encoders[col].fit_transform(train_data[col])
    test_data[col] = test_data[col].apply(lambda x: label_encoders[col].transform([x])[0]
                                          if x in label_encoders[col].classes_ 
                                          else -1)

In [8]:
# Feature scaling
scaler = StandardScaler()
features = ['Pclass', 'Fare', 'Age', 'SibSp', 'Parch', 'FamilySize', 'Title']
train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

In [9]:
# Define target and features
y = train_data['Survived']
X = train_data[features]
X_test = test_data[features]

In [10]:
# Create logistic regression model with regularization
model = LogisticRegression(penalty='l1', solver='liblinear')

In [11]:
# Define hyperparameter grid for tuning
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

In [12]:
# Create grid search object
grid_search = GridSearchCV(model, param_grid, cv=5)

In [13]:
# Fit the model with grid search
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=LogisticRegression(penalty='l1', solver='liblinear'),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10]})

In [14]:
# Get the best model
best_model = grid_search.best_estimator_

In [15]:
# Predict on the test set (no accuracy check here)
predictions = best_model.predict(X_test)

In [16]:
# Create output DataFrame
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
